# Création d'un dictionnaire qui contient un tableau pour chaque ligne de métro

## Chaque tableau affiche les stations de la ligne, leur fréquentaiton annuelle, leur fréquentation hebdomadaire moyenne sur la période 24/07/2022-11/08/2022 (période des JO mais en 2022)

In [193]:
import pandas as pd
import numpy as np
import re
import difflib

Copie du programme Nettoyage données affluence par jour + nombre de validation par jour par station

In [194]:
%config NotebookApp.iopub_data_rate_limit=100000000000.0
url = '/workspaces/metro-JO/Données importées/2022_S2_PROFIL_FER.csv'
données_validation = pd.read_csv(url, delimiter=';')

# Convertir la colonne 'pourc_validations' en float (remplacer la virgule par un point pour séparer les décimales)
données_validation['pourc_validations'] = données_validation['pourc_validations'].str.replace(',', '.').astype(float)

# Grouper par 'LIBELLE_ARRET' et 'TRNC_HORR_60', puis calculer la somme des pourc_validations pour chaque groupe
tableau_somme_pour_validations = données_validation.groupby(['LIBELLE_ARRET', 'TRNC_HORR_60'])['pourc_validations'].sum().reset_index()


# Mapping des tranches horaires à des heures spécifiques
tranche_horaire_to_hour = {
    '0H-1H': '00:00-01:00',
    '1H-2H': '01:00-02:00',
    '2H-3H': '02:00-03:00',
    '3H-4H': '03:00-04:00',
    '4H-5H': '04:00-05:00',
    '5H-6H': '05:00-06:00',
    '6H-7H': '06:00-07:00',
    '7H-8H': '07:00-08:00',
    '8H-9H': '08:00-09:00',
    '9H-10H': '09:00-10:00',
    '10H-11H': '10:00-11:00',
    '11H-12H': '11:00-12:00',
    '12H-13H': '12:00-13:00',
    '13H-14H': '13:00-14:00',
    '14H-15H': '14:00-15:00',
    '15H-16H': '15:00-16:00',
    '16H-17H': '16:00-17:00',
    '17H-18H': '17:00-18:00',
    '18H-19H': '18:00-19:00',
    '19H-20H': '19:00-20:00',
    '20H-21H': '20:00-21:00',
    '21H-22H': '21:00-22:00',
    '22H-23H': '22:00-23:00',
    '23H-24H': '23:00-00:00'
}

# Appliquer la conversion des tranches horaires au format heure
tableau_somme_pour_validations['TRNC_HORR_60'] = tableau_somme_pour_validations['TRNC_HORR_60'].map(tranche_horaire_to_hour)

# Regroupement des données en cas de doublons pour une paire 'LIBELLE_ARRET'-'TRNC_HORR_60'
grouped_data = données_validation.groupby(['LIBELLE_ARRET', 'TRNC_HORR_60'])['pourc_validations'].sum().reset_index()

# Pivotement des données
pivot_table = grouped_data.pivot(index='LIBELLE_ARRET', columns='TRNC_HORR_60', values='pourc_validations').reset_index()

# Renommage des colonnes
pivot_table.columns.name = None  # Supprimer le nom de la colonne index
pivot_table = pivot_table.rename(columns=lambda x: f"{x}" if x != 'LIBELLE_ARRET' else x)

# Sélectionner les colonnes correspondant aux tranches horaires
horaire_columns = pivot_table.columns[1:]  # Exclure la colonne 'LIBELLE_ARRET'

# Fonction pour extraire l'heure de début de chaque tranche horaire
def extract_start_hour(column):
    match = re.match(r'(\d+)H-(\d+)H', column)
    if match:
        return int(match.group(1))
    return 0

# Trier les colonnes dans l'ordre croissant des heures de début des tranches
sorted_columns = sorted(horaire_columns, key=extract_start_hour)

# Réorganiser le tableau pivoté avec les colonnes dans l'ordre chronologique
new_column_order = ['LIBELLE_ARRET'] + sorted_columns
pivot_table = pivot_table[new_column_order]

# Supprimer la colonne 'ND' du DataFrame pivot_table_ordered
pivot_table.drop(columns=['ND'], inplace=True, errors='ignore')

pivot_table['LIBELLE_ARRET'] = pivot_table['LIBELLE_ARRET'].str.strip()  # Supprime les espaces en début et fin
pivot_table['LIBELLE_ARRET'] = pivot_table['LIBELLE_ARRET'].str.lower()  # Met tout en minuscules

# Créer un dictionnaire de correspondance pour renommer les colonnes
correspondance_heures = {
    '0H-1H': '00:00-01:00', '1H-2H': '01:00-02:00', '2H-3H': '02:00-03:00', '3H-4H': '03:00-04:00',
    '4H-5H': '04:00-05:00', '5H-6H': '05:00-06:00', '6H-7H': '06:00-07:00', '7H-8H': '07:00-08:00',
    '8H-9H': '08:00-09:00', '9H-10H': '09:00-10:00', '10H-11H': '10:00-11:00', '11H-12H': '11:00-12:00',
    '12H-13H': '12:00-13:00', '13H-14H': '13:00-14:00', '14H-15H': '14:00-15:00', '15H-16H': '15:00-16:00',
    '16H-17H': '16:00-17:00', '17H-18H': '17:00-18:00', '18H-19H': '18:00-19:00', '19H-20H': '19:00-20:00',
    '20H-21H': '20:00-21:00', '21H-22H': '21:00-22:00', '22H-23H': '22:00-23:00', '23H-0H': '23:00-00:00'
}

# Renommer les colonnes en utilisant le dictionnaire de correspondance
pivot_table.rename(columns=correspondance_heures, inplace=True)

/tmp/ipykernel_2347/1420814720.py:3: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  données_validation = pd.read_csv(url, delimiter=';')


Importation ableau avec station, ligne, correspondances... (Données disponibles sur le site https://data.ratp.fr)

In [195]:
%pip install openpyxl
tableau_fréquentation = pd.read_excel('https://data.ratp.fr/api/explore/v2.1/catalog/datasets/trafic-annuel-entrant-par-station-du-reseau-ferre-2021/exports/xlsx?lang=fr&timezone=Europe%2FBerlin&use_labels=true')

tableau_fréquentation = tableau_fréquentation[tableau_fréquentation['Réseau'] == 'Métro']
tableau_fréquentation = tableau_fréquentation.drop_duplicates(subset='Station')


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Nettoyage du tableau

In [196]:
colonnes_a_supprimer = ['Réseau', 'Ville', 'Arrondissement pour Paris']

tableau_fréquentation = tableau_fréquentation.drop(colonnes_a_supprimer, axis=1)

# Liste des colonnes à modifier
colonnes_correspondance = ['Correspondance_1', 'Correspondance_2', 'Correspondance_3', 'Correspondance_4', 'Correspondance_5']

# Convertir les colonnes spécifiées en type chaîne (si elles ne le sont pas déjà)
tableau_fréquentation[colonnes_correspondance] = tableau_fréquentation[colonnes_correspondance].astype(str)

# Supprimer les lettres 'A' et 'B' des colonnes spécifiées dans tableau_fréquentation
for colonne in colonnes_correspondance:
    tableau_fréquentation[colonne] = tableau_fréquentation[colonne].str.replace('A', '').str.replace('B', '')

print(tableau_fréquentation)

     Rang               Station    Trafic Correspondance_1 Correspondance_2  \
0       3          GARE DE LYON  28640475                1               14   
1      12            BELLEVILLE   7314438                2               11   
2      32                 OPERA   5193831                3                7   
3      58  SAINT-MANDE-TOURELLE   3944640                1              nan   
4      59                 HOCHE   3928404                5              nan   
..    ...                   ...       ...              ...              ...   
350   272         SULLY-MORLAND   1124169                7              nan   
351   275                EUROPE   1106459                3              nan   
352   283          CAMPO-FORMIO   1016150                5              nan   
353   298         SAINT-FARGEAU    472258             3bis              nan   
354   300       PORTE D'AUTEUIL    375748               10              nan   

    Correspondance_3 Correspondance_4 Correspondanc

In [197]:
# Ajouter la colonne 'nb_correspondance'
tableau_fréquentation['nb_correspondance'] = tableau_fréquentation[colonnes_correspondance].apply(lambda row: sum(pd.notna(value) and value != 'nan' for value in row), axis=1)
print(tableau_fréquentation)

     Rang               Station    Trafic Correspondance_1 Correspondance_2  \
0       3          GARE DE LYON  28640475                1               14   
1      12            BELLEVILLE   7314438                2               11   
2      32                 OPERA   5193831                3                7   
3      58  SAINT-MANDE-TOURELLE   3944640                1              nan   
4      59                 HOCHE   3928404                5              nan   
..    ...                   ...       ...              ...              ...   
350   272         SULLY-MORLAND   1124169                7              nan   
351   275                EUROPE   1106459                3              nan   
352   283          CAMPO-FORMIO   1016150                5              nan   
353   298         SAINT-FARGEAU    472258             3bis              nan   
354   300       PORTE D'AUTEUIL    375748               10              nan   

    Correspondance_3 Correspondance_4 Correspondanc

In [198]:
# Diviser les valeurs de la colonne "Trafic" par les valeurs de la colonne "nb_correspondance"
tableau_fréquentation['Trafic'] = tableau_fréquentation['Trafic'] / tableau_fréquentation['nb_correspondance']
print(tableau_fréquentation)

     Rang               Station     Trafic Correspondance_1 Correspondance_2  \
0       3          GARE DE LYON  9546825.0                1               14   
1      12            BELLEVILLE  3657219.0                2               11   
2      32                 OPERA  1731277.0                3                7   
3      58  SAINT-MANDE-TOURELLE  3944640.0                1              nan   
4      59                 HOCHE  3928404.0                5              nan   
..    ...                   ...        ...              ...              ...   
350   272         SULLY-MORLAND  1124169.0                7              nan   
351   275                EUROPE  1106459.0                3              nan   
352   283          CAMPO-FORMIO  1016150.0                5              nan   
353   298         SAINT-FARGEAU   472258.0             3bis              nan   
354   300       PORTE D'AUTEUIL   375748.0               10              nan   

    Correspondance_3 Correspondance_4 C

Création d'un tableau avec la moyenne sur la période des JO en 2022 du nombre de validation hebdomadaire à chaque station

In [199]:
#Nettoyage de données sur le nombre de validations: on obtient le nombre de validation moyen par jour sur 1 mois précis

import pandas as pd
%config NotebookApp.iopub_data_rate_limit=100000000000.0
url2 = '/workspaces/metro-JO/Données importées/2022_S2_NB_FER.csv'
données_validation2= pd.read_csv(url2)
données_validation2 = pd.read_csv(url2, delimiter=';')

# Exemple avec gestion de formats flexibles
données_validation2['JOUR'] = pd.to_datetime(données_validation2['JOUR'], errors='coerce', format="%d/%m/%Y")
donnees_mai = données_validation2.loc[(données_validation2['JOUR'].dt.month == 5)]
donnees_sum = donnees_mai.groupby('LIBELLE_ARRET')['NB_VALD'].mean().reset_index()

# Conversion de la colonne 'JOUR' en datetime
données_validation2['JOUR'] = pd.to_datetime(données_validation2['JOUR'], format='%d/%m/%Y')

# Filtrer les données pour les dates spécifiques
donnees_filtrees = données_validation2[(données_validation2['JOUR'] >= '2022-07-24') & (données_validation2['JOUR'] <= '2022-08-11')]

# Grouper par 'JOUR', 'LIBELLE_ARRET' et 'CATEGORIE_TITRE' et calculer la somme des 'NB_VALD'
donnees_aggreg = donnees_filtrees.groupby(['JOUR', 'LIBELLE_ARRET', 'CATEGORIE_TITRE'])['NB_VALD'].sum().reset_index()

# Conversion de la colonne 'JOUR' en datetime si ce n'est pas déjà fait
données_validation2['JOUR'] = pd.to_datetime(données_validation2['JOUR'], format='%d/%m/%Y')

# Filtrage des dates spécifiques
donnees_filtrees = données_validation2[(données_validation2['JOUR'] >= '2022-07-24') & (données_validation2['JOUR'] <= '2022-08-11')]

# Somme de NB_VALD par JOUR et par LIBELLE_ARRET
somme_par_jour_arret = donnees_filtrees.groupby(['JOUR', 'LIBELLE_ARRET'])['NB_VALD'].sum().reset_index()

moyenne_par_arret = somme_par_jour_arret.groupby('LIBELLE_ARRET')['NB_VALD'].mean().reset_index()

# Normalisation des libellés
moyenne_par_arret['LIBELLE_ARRET'] = moyenne_par_arret['LIBELLE_ARRET'].str.lower().str.strip()

# Somme des valeurs de NB_VALD pour les libellés normalisés
moyenne_par_arret = moyenne_par_arret.groupby('LIBELLE_ARRET')['NB_VALD'].sum().reset_index()

# Suppression des doublons
moyenne_par_arret = moyenne_par_arret.drop_duplicates(subset=['LIBELLE_ARRET'])

display(moyenne_par_arret)

,LIBELLE_ARRET,NB_VALD
0,abbesses,2277.315789
1,ablon,1101.105263
2,acheres grand,51.388889
3,acheres ville,3012.052632
4,aeroport cdg1,5050.947368
...,...,...
733,vosves,14.894737
734,vulaines sur se,42.894737
735,wagram,3268.157895
736,yerres,2743.000000


Création d'un dictionnaire contenant les fréquentations de chaque station de chaque ligne 

In [200]:
from IPython.display import display

numéro = [1, 2, 3, '3bis', 4, 5, 6, 7, '7bis', 8, 9, 10, 11, 12, 13, 14]

# Création des tableaux pour chaque élément de numéro
tableaux = {}

colonnes_correspondance = ['Correspondance_1', 'Correspondance_2', 'Correspondance_3', 'Correspondance_4', 'Correspondance_5']

for k in numéro:
    # Convertir k en string car il peut contenir des valeurs non numériques
    k_str = str(k)
    
    # Filtrer les lignes où la valeur k est présente dans les colonnes spécifiées
    filtered_rows = tableau_fréquentation[tableau_fréquentation[colonnes_correspondance].apply(lambda x: k_str in str(x.values), axis=1)]
    
    # Supprimer les doublons dans la colonne 'Station'
    filtered_rows = filtered_rows.drop_duplicates(subset='Station')
    
    # Stocker les lignes filtrées dans un tableau nommé "ligne numéro[k]"
    tableaux[f'ligne numéro[{k_str}]'] = filtered_rows

# Parcourir chaque tableau dans le dictionnaire et supprimer les colonnes spécifiées
for tableau_nom, tableau in tableaux.items():
    tableaux[tableau_nom] = tableau.drop(columns=colonnes_correspondance)
    tableau = tableau.drop_duplicates(subset='Station')
    tableaux[tableau_nom] = tableau.drop_duplicates(subset='Station')  # Ajoutez cette ligne pour supprimer les doublons de la colonne 'Station'


In [201]:

# Parcourir chaque tableau dans le dictionnaire
for tableau_nom, tableau in tableaux.items():
    # Calculer la fréquentation totale pour chaque tableau
    fréquentation_totale_ligne = tableau['Trafic'].sum()

    # Ajouter la variable fréquentation_totale à chaque tableau
    tableaux[tableau_nom]['fréquentation_totale_ligne'] = fréquentation_totale_ligne
    tableau['Part du trafic de la station par rapport à toute la ligne'] = tableau['Trafic'] / tableau['fréquentation_totale_ligne']

# Supposons que la clé pour la correspondance soit 'LIBELLE_ARRET' dans le tableau moyenne_par_arret
cle_correspondance = 'LIBELLE_ARRET'

# Nettoyage des espaces et uniformisation de la casse
moyenne_par_arret['LIBELLE_ARRET'] = moyenne_par_arret['LIBELLE_ARRET'].str.strip().str.lower()
for tableau_nom, tableau in tableaux.items():
    tableau['Station'] = tableau['Station'].str.strip().str.lower()

    # Fusion en utilisant les noms de stations nettoyés
    merged_table = pd.merge(tableau, moyenne_par_arret, left_on='Station', right_on='LIBELLE_ARRET', how='left')
    
    # Renommer la colonne NB_VALD de moyenne_par_arret
    merged_table = merged_table.rename(columns={'NB_VALD': 'fréquentation moyenne hors JO'})
    
    tableaux[tableau_nom] = merged_table

In [202]:
#compléter avec les valeurs manquantes

# Méthode pour trouver la correspondance la plus proche
def find_closest_match(target, candidates):
    closest_match = difflib.get_close_matches(target, candidates, n=1, cutoff=0.4)  # Ajustez le seuil selon vos besoins
    return closest_match[0] if closest_match else None

for k in numéro:
    k_str = str(k)
    # Accéder aux tableaux spécifiques
    current_table = tableaux[f'ligne numéro[{k_str}]']
    
    # Filtrer les lignes avec des valeurs manquantes dans la colonne 'LIBELLE_ARRET'
    missing_data = current_table[current_table['LIBELLE_ARRET'].isnull()]
    
    # Remplir les valeurs manquantes
    for index, row in missing_data.iterrows():
        closest_match = find_closest_match(row['Station'], moyenne_par_arret['LIBELLE_ARRET'])
        
        if closest_match:
            tableaux[f'ligne numéro[{k_str}]'].at[index, 'LIBELLE_ARRET'] = closest_match



In [203]:
for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Fusionner avec le tableau moyenne_par_arret pour obtenir les valeurs NB_VALD associées
    merged_with_nb_vald = pd.merge(current_table, moyenne_par_arret[['LIBELLE_ARRET', 'NB_VALD']],
                                   left_on='LIBELLE_ARRET', right_on='LIBELLE_ARRET', how='left')
    
    # Renommer la colonne NB_VALD pour plus de clarté
    merged_with_nb_vald = merged_with_nb_vald.rename(columns={'NB_VALD': 'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station'})

    # Mettre à jour le tableau dans le dictionnaire
    tableaux[f'ligne numéro[{k_str}]'] = merged_with_nb_vald

for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Fusionner les données avec le tableau contenant les nouvelles colonnes
    merged_table = pd.merge(current_table, pivot_table, on='LIBELLE_ARRET', how='left')

    # Mettre à jour le tableau dans le dictionnaire avec les nouvelles colonnes fusionnées
    tableaux[f'ligne numéro[{k_str}]'] = merged_table

In [204]:

for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Supprimer les doublons de stations
    current_table = current_table.drop_duplicates(subset='Station')

    # Mettre à jour le tableau dans le dictionnaire
    tableaux[f'ligne numéro[{k_str}]'] = current_table


In [205]:
for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Supprimer les colonnes 'fréquentation moyenne hors JO' et 'LIBELLE_ARRET' si elles existent
    columns_to_drop = ['fréquentation moyenne hors JO', 'LIBELLE_ARRET']
    current_table = current_table.drop(columns=columns_to_drop, errors='ignore')

    # Mettre à jour le tableau dans le dictionnaire
    tableaux[f'ligne numéro[{k_str}]'] = current_table

In [206]:
for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Vérifier si les colonnes existent avant de créer la nouvelle colonne
    if 'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station' in current_table.columns and 'nb_correspondance' in current_table.columns:
        # Calcul de la nouvelle colonne
        current_table['moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne'] = current_table['moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station'] / current_table['nb_correspondance']

        # Mettre à jour le tableau dans le dictionnaire
        tableaux[f'ligne numéro[{k_str}]'] = current_table

for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Vérifier si les colonnes existent avant de créer la nouvelle colonne
    if 'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station' in current_table.columns and 'nb_correspondance' in current_table.columns:
        # Calcul de la nouvelle colonne
        current_table['moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction'] = current_table['moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station'] / current_table['nb_correspondance'] / 2

        # Mettre à jour le tableau dans le dictionnaire
        tableaux[f'ligne numéro[{k_str}]'] = current_table

for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Vérifier si la colonne existe avant de créer la nouvelle colonne
    if 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne' in current_table.columns:
        # Calculer la somme de la colonne spécifique
        current_table['nombre_validations_totales_pour_la_ligne'] = current_table['moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne'].sum()

        # Mettre à jour le tableau dans le dictionnaire
        tableaux[f'ligne numéro[{k_str}]'] = current_table


Tableau 'ligne numéro[1]' avec la nouvelle colonne 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction' :


,Rang,Station,Trafic,Correspondance_1,Correspondance_2,Correspondance_3,Correspondance_4,Correspondance_5,nb_correspondance,fréquentation_totale_ligne,...,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nombre_validations_totales_pour_la_ligne
0,3,gare de lyon,9546825.0,1,14,,nan,nan,3,2.985900e+08,...,57.99,60.21,38.73,27.60,17.40,10.24,4.80,22012.649123,11006.324561,756681.396934
2,12,belleville,3657219.0,2,11,nan,nan,nan,2,2.985900e+08,...,41.62,37.49,32.01,21.30,14.72,13.35,11.97,8305.815789,4152.907895,756681.396934
3,58,saint-mande-tourelle,3944640.0,1,nan,nan,nan,nan,1,2.985900e+08,...,44.98,47.84,34.26,19.04,10.38,8.05,5.99,8727.684211,4363.842105,756681.396934
4,61,bercy,1942106.0,6,14,nan,nan,nan,2,2.985900e+08,...,104.32,67.23,40.18,13.57,0.11,14.23,0.09,4511.973684,2255.986842,756681.396934
6,74,saint-denis-universite,3569990.0,13,nan,nan,nan,nan,1,2.985900e+08,...,31.51,27.77,18.64,13.60,8.89,6.17,4.36,8049.368421,4024.684211,756681.396934
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,234,michel-ange-auteuil,756025.0,9,10,nan,nan,nan,2,2.985900e+08,...,43.35,40.46,35.29,21.50,11.78,11.57,9.71,1449.815789,724.907895,756681.396934
124,238,notre-dame-des-champs,1487256.0,12,nan,nan,nan,nan,1,2.985900e+08,...,51.64,51.20,38.02,25.48,14.76,14.98,12.61,1661.578947,830.789474,756681.396934
125,239,malakoff-rue etienne dolet,1450451.0,13,nan,nan,nan,nan,1,2.985900e+08,...,43.10,41.24,28.00,15.65,8.94,6.14,4.88,3242.105263,1621.052632,756681.396934
126,243,michel-ange-molitor,710276.0,9,10,nan,nan,nan,2,2.985900e+08,...,42.74,38.67,35.84,19.83,11.89,10.72,13.92,1290.657895,645.328947,756681.396934


Tableau 'ligne numéro[2]' avec la nouvelle colonne 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction' :


,Rang,Station,Trafic,Correspondance_1,Correspondance_2,Correspondance_3,Correspondance_4,Correspondance_5,nb_correspondance,fréquentation_totale_ligne,...,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nombre_validations_totales_pour_la_ligne
0,12,belleville,3.657219e+06,2,11,nan,nan,nan,2,1.157136e+08,...,41.62,37.49,32.01,21.30,14.72,13.35,11.97,8305.815789,4152.907895,297028.650658
1,104,colonel fabien,3.043606e+06,2,nan,nan,nan,nan,1,1.157136e+08,...,45.10,41.33,34.81,22.89,15.25,12.20,10.86,6090.315789,3045.157895,297028.650658
2,118,menilmontant,2.847264e+06,2,nan,nan,nan,nan,1,1.157136e+08,...,37.85,37.05,33.65,24.10,18.15,17.53,17.79,5824.315789,2912.157895,297028.650658
3,163,blanche,2.167570e+06,2,nan,nan,nan,nan,1,1.157136e+08,...,41.29,41.39,36.88,27.88,21.15,27.09,24.68,4236.421053,2118.210526,297028.650658
4,242,rue du bac,1.423364e+06,12,nan,nan,nan,nan,1,1.157136e+08,...,57.77,57.82,48.87,23.13,15.46,12.09,11.93,2642.842105,1321.421053,297028.650658
5,256,philippe auguste,1.262653e+06,2,nan,nan,nan,nan,1,1.157136e+08,...,49.12,42.83,32.67,21.55,13.32,12.08,9.82,2473.842105,1236.921053,297028.650658
6,268,monceau,1.148677e+06,2,nan,nan,nan,nan,1,1.157136e+08,...,62.67,65.78,47.89,22.54,11.59,7.52,5.00,2529.789474,1264.894737,297028.650658
7,288,rennes,8.613340e+05,12,nan,nan,nan,nan,1,1.157136e+08,...,58.20,56.48,44.05,23.63,10.44,10.24,8.72,1546.052632,773.026316,297028.650658
8,4,montparnasse-bienvenue,5.101806e+06,4,6,12,13.0,nan,4,1.157136e+08,...,55.50,62.72,49.65,31.05,21.54,16.83,13.49,16778.171053,8389.085526,297028.650658
10,27,madeleine,1.776976e+06,8,12,14,nan,nan,3,1.157136e+08,...,65.73,71.49,60.81,23.76,11.64,11.65,10.10,4301.473684,2150.736842,297028.650658


Tableau 'ligne numéro[3]' avec la nouvelle colonne 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction' :


...

,Rang,Station,Trafic,Correspondance_1,Correspondance_2,Correspondance_3,Correspondance_4,Correspondance_5,nb_correspondance,fréquentation_totale_ligne,...,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nombre_validations_totales_pour_la_ligne
0,242,rue du bac,1.423364e+06,12,nan,nan,nan,nan,1,6.067441e+07,...,57.77,57.82,48.87,23.13,15.46,12.09,11.93,2642.842105,1321.421053,152539.641886
1,288,rennes,8.613340e+05,12,nan,nan,nan,nan,1,6.067441e+07,...,58.20,56.48,44.05,23.63,10.44,10.24,8.72,1546.052632,773.026316,152539.641886
2,4,montparnasse-bienvenue,5.101806e+06,4,6,12,13.0,nan,4,6.067441e+07,...,55.50,62.72,49.65,31.05,21.54,16.83,13.49,16778.171053,8389.085526,152539.641886
4,27,madeleine,1.776976e+06,8,12,14,nan,nan,3,6.067441e+07,...,65.73,71.49,60.81,23.76,11.64,11.65,10.10,4301.473684,2150.736842,152539.641886
5,88,sevres-babylone,1.696252e+06,10,12,nan,nan,nan,2,6.067441e+07,...,59.22,66.54,57.85,35.27,12.25,9.71,8.96,3270.052632,1635.026316,152539.641886
6,115,mairie d'issy,2.874138e+06,12,nan,nan,nan,nan,1,6.067441e+07,...,38.97,38.91,30.74,16.37,8.48,6.66,5.20,6616.526316,3308.263158,152539.641886
7,188,notre-dame-de-lorette,1.893828e+06,12,nan,nan,nan,nan,1,6.067441e+07,...,44.18,49.69,39.57,25.65,17.51,18.02,18.33,3949.578947,1974.789474,152539.641886
8,93,porte de versailles,3.268157e+06,12,nan,nan,nan,nan,1,6.067441e+07,...,60.08,62.00,53.19,32.02,19.18,19.01,18.49,15023.947368,7511.973684,152539.641886
9,136,vaugirard,2.482886e+06,12,nan,nan,nan,nan,1,6.067441e+07,...,44.22,41.04,34.89,19.16,10.52,9.18,7.66,4768.736842,2384.368421,152539.641886
10,139,marx dormoy,2.425928e+06,12,nan,nan,nan,nan,1,6.067441e+07,...,43.01,38.88,31.06,21.78,14.49,10.47,8.05,5344.315789,2672.157895,152539.641886


Tableau 'ligne numéro[13]' avec la nouvelle colonne 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction' :


,Rang,Station,Trafic,Correspondance_1,Correspondance_2,Correspondance_3,Correspondance_4,Correspondance_5,nb_correspondance,fréquentation_totale_ligne,...,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nombre_validations_totales_pour_la_ligne
0,74,saint-denis-universite,3569990.0,13,nan,nan,nan,nan,1,83039404.3,...,31.51,27.77,18.64,13.60,8.89,6.17,4.36,8049.368421,4024.684211,196412.634211
1,122,les courtilles,2802689.0,13,nan,nan,nan,nan,1,83039404.3,...,35.24,31.76,23.70,16.18,10.77,7.34,4.88,8152.789474,4076.394737,196412.634211
2,125,porte de saint-ouen,2710638.0,13,nan,nan,nan,nan,1,83039404.3,...,42.89,40.38,34.53,23.47,16.62,12.24,9.27,6262.368421,3131.184211,196412.634211
3,4,montparnasse-bienvenue,5101806.0,4,6,12,13.0,nan,4,83039404.3,...,55.50,62.72,49.65,31.05,21.54,16.83,13.49,16778.171053,8389.085526,196412.634211
5,29,porte de clichy,2639248.5,13,14,nan,nan,nan,2,83039404.3,...,92.43,88.85,71.86,46.94,32.62,24.38,18.66,6307.552632,3153.776316,196412.634211
6,33,place clichy,2580966.0,2,13,nan,nan,nan,2,83039404.3,...,41.68,40.48,38.04,26.06,20.67,21.62,21.26,5461.368421,2730.684211,196412.634211
7,87,porte de vanves,3395358.0,13,nan,nan,nan,nan,1,83039404.3,...,41.18,39.19,30.95,21.34,13.95,10.18,8.11,7711.315789,3855.657895,196412.634211
8,162,pernety,2173567.0,13,nan,nan,nan,nan,1,83039404.3,...,34.66,34.29,31.53,21.05,13.67,12.71,10.96,4404.263158,2202.131579,196412.634211
9,230,carrefour pleyel,1535528.0,13,nan,nan,nan,nan,1,83039404.3,...,57.68,45.30,28.87,16.22,10.19,8.39,6.78,3933.789474,1966.894737,196412.634211
10,64,gabriel peri,3790573.0,13,nan,nan,nan,nan,1,83039404.3,...,38.88,38.49,29.48,18.40,11.34,7.23,5.40,9837.842105,4918.921053,196412.634211


Tableau 'ligne numéro[14]' avec la nouvelle colonne 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction' :


,Rang,Station,Trafic,Correspondance_1,Correspondance_2,Correspondance_3,Correspondance_4,Correspondance_5,nb_correspondance,fréquentation_totale_ligne,...,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nombre_validations_totales_pour_la_ligne
0,3,gare de lyon,9546825.0,1,14,,nan,nan,3,55491326.1,...,57.99,60.21,38.73,27.60,17.40,10.24,4.80,22012.649123,11006.324561,131269.920635
2,61,bercy,1942106.0,6,14,nan,nan,nan,2,55491326.1,...,104.32,67.23,40.18,13.57,0.11,14.23,0.09,4511.973684,2255.986842,131269.920635
4,27,madeleine,1776976.0,8,12,14,nan,nan,3,55491326.1,...,65.73,71.49,60.81,23.76,11.64,11.65,10.10,4301.473684,2150.736842,131269.920635
5,29,porte de clichy,2639248.5,13,14,nan,nan,nan,2,55491326.1,...,92.43,88.85,71.86,46.94,32.62,24.38,18.66,6307.552632,3153.776316,131269.920635
6,110,cour saint-emilion,2985122.0,14,nan,nan,nan,nan,1,55491326.1,...,61.54,60.91,45.62,29.16,25.69,15.63,12.76,3173.444444,1586.722222,131269.920635
7,6,bibliotheque,11104474.0,14,nan,nan,nan,nan,1,55491326.1,...,54.02,56.83,44.83,26.99,18.84,11.58,8.76,6088.000000,3044.000000,131269.920635
8,10,chatelet,1670158.8,1,4,7,11.0,14.0,5,55491326.1,...,114.31,122.78,105.60,70.85,51.84,47.85,44.60,17555.905263,8777.952632,131269.920635
9,31,olympiades,5214595.0,14,nan,nan,nan,nan,1,55491326.1,...,47.26,41.98,31.78,20.50,13.19,6.90,4.47,12997.142857,6498.571429,131269.920635
10,2,saint-lazare,6625676.8,3,9,12,13.0,14.0,5,55491326.1,...,57.36,63.81,51.34,34.47,21.65,16.09,13.86,25432.673684,12716.336842,131269.920635
12,39,mairie de saint-ouen,2415405.0,13,14,nan,nan,nan,2,55491326.1,...,93.80,89.63,63.45,40.80,24.30,16.35,13.66,6008.868421,3004.434211,131269.920635


In [207]:
# Définir le nouvel ordre des colonnes
nouvel_ordre_colonnes = [
    'Station',
    'Trafic',
    'Part du trafic de la station par rapport à toute la ligne',
    'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station',
    'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne',
    'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction', 'nb_correspondance',
    'Correspondance_1',
    'Correspondance_2',
    'Correspondance_3',
    'Correspondance_4',
    'Correspondance_5',
    'nombre_validations_totales_pour_la_ligne',
    'fréquentation_totale_ligne',
    '00:00-01:00',
    '01:00-02:00',
    '02:00-03:00',
    '03:00-04:00',
    '04:00-05:00',
    '05:00-06:00',
    '06:00-07:00',
    '07:00-08:00',
    '08:00-09:00',
    '09:00-10:00',
    '10:00-11:00',
    '11:00-12:00',
    '12:00-13:00',
    '13:00-14:00',
    '14:00-15:00',
    '15:00-16:00',
    '16:00-17:00',
    '17:00-18:00',
    '18:00-19:00',
    '19:00-20:00',
    '20:00-21:00',
    '21:00-22:00',
    '22:00-23:00',
    '23:00-00:00',
    'Rang'
]

# Appliquer le nouvel ordre des colonnes à chaque tableau
for tableau_nom, tableau in tableaux.items():
    tableaux[tableau_nom] = tableau[nouvel_ordre_colonnes]

# Vérifier que rien n'a été oublié
print("Colonnes disponibles :")
print(tableau.columns)
for tableau_nom, tableau in tableaux.items():
    print(f"Tableau: {tableau_nom}")
    
    # Réorganiser les colonnes selon le nouvel ordre
    tableau = tableau[nouvel_ordre_colonnes]
    
    display(tableau)
    print("\n" + "=" * 50 + "\n")

Colonnes disponibles :
Index(['Rang', 'Station', 'Trafic', 'Correspondance_1', 'Correspondance_2',
       'Correspondance_3', 'Correspondance_4', 'Correspondance_5',
       'nb_correspondance', 'fréquentation_totale_ligne',
       'Part du trafic de la station par rapport à toute la ligne',
       'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station',
       '00:00-01:00', '01:00-02:00', '02:00-03:00', '03:00-04:00',
       '04:00-05:00', '05:00-06:00', '06:00-07:00', '07:00-08:00',
       '08:00-09:00', '09:00-10:00', '10:00-11:00', '11:00-12:00',
       '12:00-13:00', '13:00-14:00', '14:00-15:00', '15:00-16:00',
       '16:00-17:00', '17:00-18:00', '18:00-19:00', '19:00-20:00',
       '20:00-21:00', '21:00-22:00', '22:00-23:00', '23:00-00:00',
       'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne',
       'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction',
       'nombre_validations_totales_pour_la_ligne'],
      dtype='object')
Tableau: ligne numéro[1]


,Station,Trafic,Part du trafic de la station par rapport à toute la ligne,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nb_correspondance,Correspondance_1,Correspondance_2,Correspondance_3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,gare de lyon,9546825.0,0.031973,66037.947368,22012.649123,11006.324561,3,1,14,,...,34.76,45.22,57.99,60.21,38.73,27.60,17.40,10.24,4.80,3
2,belleville,3657219.0,0.012248,16611.631579,8305.815789,4152.907895,2,2,11,nan,...,38.84,39.88,41.62,37.49,32.01,21.30,14.72,13.35,11.97,12
3,saint-mande-tourelle,3944640.0,0.013211,8727.684211,8727.684211,4363.842105,1,1,nan,nan,...,35.69,37.54,44.98,47.84,34.26,19.04,10.38,8.05,5.99,58
4,bercy,1942106.0,0.006504,9023.947368,4511.973684,2255.986842,2,6,14,nan,...,2.37,109.81,104.32,67.23,40.18,13.57,0.11,14.23,0.09,61
6,saint-denis-universite,3569990.0,0.011956,8049.368421,8049.368421,4024.684211,1,13,nan,nan,...,30.33,28.90,31.51,27.77,18.64,13.60,8.89,6.17,4.36,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,michel-ange-auteuil,756025.0,0.002532,2899.631579,1449.815789,724.907895,2,9,10,nan,...,37.63,40.86,43.35,40.46,35.29,21.50,11.78,11.57,9.71,234
124,notre-dame-des-champs,1487256.0,0.004981,1661.578947,1661.578947,830.789474,1,12,nan,nan,...,36.70,46.97,51.64,51.20,38.02,25.48,14.76,14.98,12.61,238
125,malakoff-rue etienne dolet,1450451.0,0.004858,3242.105263,3242.105263,1621.052632,1,13,nan,nan,...,33.51,34.59,43.10,41.24,28.00,15.65,8.94,6.14,4.88,239
126,michel-ange-molitor,710276.0,0.002379,2581.315789,1290.657895,645.328947,2,9,10,nan,...,37.02,39.89,42.74,38.67,35.84,19.83,11.89,10.72,13.92,243




Tableau: ligne numéro[2]


,Station,Trafic,Part du trafic de la station par rapport à toute la ligne,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nb_correspondance,Correspondance_1,Correspondance_2,Correspondance_3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,belleville,3.657219e+06,0.031606,16611.631579,8305.815789,4152.907895,2,2,11,nan,...,38.84,39.88,41.62,37.49,32.01,21.30,14.72,13.35,11.97,12
1,colonel fabien,3.043606e+06,0.026303,6090.315789,6090.315789,3045.157895,1,2,nan,nan,...,35.77,39.02,45.10,41.33,34.81,22.89,15.25,12.20,10.86,104
2,menilmontant,2.847264e+06,0.024606,5824.315789,5824.315789,2912.157895,1,2,nan,nan,...,32.69,34.55,37.85,37.05,33.65,24.10,18.15,17.53,17.79,118
3,blanche,2.167570e+06,0.018732,4236.421053,4236.421053,2118.210526,1,2,nan,nan,...,34.19,37.46,41.29,41.39,36.88,27.88,21.15,27.09,24.68,163
4,rue du bac,1.423364e+06,0.012301,2642.842105,2642.842105,1321.421053,1,12,nan,nan,...,40.07,50.13,57.77,57.82,48.87,23.13,15.46,12.09,11.93,242
5,philippe auguste,1.262653e+06,0.010912,2473.842105,2473.842105,1236.921053,1,2,nan,nan,...,36.68,41.25,49.12,42.83,32.67,21.55,13.32,12.08,9.82,256
6,monceau,1.148677e+06,0.009927,2529.789474,2529.789474,1264.894737,1,2,nan,nan,...,39.02,48.87,62.67,65.78,47.89,22.54,11.59,7.52,5.00,268
7,rennes,8.613340e+05,0.007444,1546.052632,1546.052632,773.026316,1,12,nan,nan,...,44.08,52.01,58.20,56.48,44.05,23.63,10.44,10.24,8.72,288
8,montparnasse-bienvenue,5.101806e+06,0.044090,67112.684211,16778.171053,8389.085526,4,4,6,12,...,26.18,36.84,55.50,62.72,49.65,31.05,21.54,16.83,13.49,4
10,madeleine,1.776976e+06,0.015357,12904.421053,4301.473684,2150.736842,3,8,12,14,...,43.85,53.91,65.73,71.49,60.81,23.76,11.64,11.65,10.10,27




Tableau: ligne numéro[3]


...

,Station,Trafic,Part du trafic de la station par rapport à toute la ligne,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nb_correspondance,Correspondance_1,Correspondance_2,Correspondance_3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,rue du bac,1.423364e+06,0.023459,2642.842105,2642.842105,1321.421053,1,12,nan,nan,...,40.07,50.13,57.77,57.82,48.87,23.13,15.46,12.09,11.93,242
1,rennes,8.613340e+05,0.014196,1546.052632,1546.052632,773.026316,1,12,nan,nan,...,44.08,52.01,58.20,56.48,44.05,23.63,10.44,10.24,8.72,288
2,montparnasse-bienvenue,5.101806e+06,0.084085,67112.684211,16778.171053,8389.085526,4,4,6,12,...,26.18,36.84,55.50,62.72,49.65,31.05,21.54,16.83,13.49,4
4,madeleine,1.776976e+06,0.029287,12904.421053,4301.473684,2150.736842,3,8,12,14,...,43.85,53.91,65.73,71.49,60.81,23.76,11.64,11.65,10.10,27
5,sevres-babylone,1.696252e+06,0.027957,6540.105263,3270.052632,1635.026316,2,10,12,nan,...,39.67,51.13,59.22,66.54,57.85,35.27,12.25,9.71,8.96,88
6,mairie d'issy,2.874138e+06,0.047370,6616.526316,6616.526316,3308.263158,1,12,nan,nan,...,34.97,36.23,38.97,38.91,30.74,16.37,8.48,6.66,5.20,115
7,notre-dame-de-lorette,1.893828e+06,0.031213,3949.578947,3949.578947,1974.789474,1,12,nan,nan,...,35.01,35.65,44.18,49.69,39.57,25.65,17.51,18.02,18.33,188
8,porte de versailles,3.268157e+06,0.053864,15023.947368,15023.947368,7511.973684,1,12,nan,nan,...,39.11,50.70,60.08,62.00,53.19,32.02,19.18,19.01,18.49,93
9,vaugirard,2.482886e+06,0.040921,4768.736842,4768.736842,2384.368421,1,12,nan,nan,...,37.27,41.28,44.22,41.04,34.89,19.16,10.52,9.18,7.66,136
10,marx dormoy,2.425928e+06,0.039983,5344.315789,5344.315789,2672.157895,1,12,nan,nan,...,36.82,37.68,43.01,38.88,31.06,21.78,14.49,10.47,8.05,139




Tableau: ligne numéro[13]


,Station,Trafic,Part du trafic de la station par rapport à toute la ligne,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nb_correspondance,Correspondance_1,Correspondance_2,Correspondance_3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,saint-denis-universite,3569990.0,0.042992,8049.368421,8049.368421,4024.684211,1,13,nan,nan,...,30.33,28.90,31.51,27.77,18.64,13.60,8.89,6.17,4.36,74
1,les courtilles,2802689.0,0.033751,8152.789474,8152.789474,4076.394737,1,13,nan,nan,...,30.03,31.76,35.24,31.76,23.70,16.18,10.77,7.34,4.88,122
2,porte de saint-ouen,2710638.0,0.032643,6262.368421,6262.368421,3131.184211,1,13,nan,nan,...,34.06,38.01,42.89,40.38,34.53,23.47,16.62,12.24,9.27,125
3,montparnasse-bienvenue,5101806.0,0.061438,67112.684211,16778.171053,8389.085526,4,4,6,12,...,26.18,36.84,55.50,62.72,49.65,31.05,21.54,16.83,13.49,4
5,porte de clichy,2639248.5,0.031783,12615.105263,6307.552632,3153.776316,2,13,14,nan,...,67.67,75.58,92.43,88.85,71.86,46.94,32.62,24.38,18.66,29
6,place clichy,2580966.0,0.031081,10922.736842,5461.368421,2730.684211,2,2,13,nan,...,34.37,37.23,41.68,40.48,38.04,26.06,20.67,21.62,21.26,33
7,porte de vanves,3395358.0,0.040889,7711.315789,7711.315789,3855.657895,1,13,nan,nan,...,32.51,35.64,41.18,39.19,30.95,21.34,13.95,10.18,8.11,87
8,pernety,2173567.0,0.026175,4404.263158,4404.263158,2202.131579,1,13,nan,nan,...,33.91,33.31,34.66,34.29,31.53,21.05,13.67,12.71,10.96,162
9,carrefour pleyel,1535528.0,0.018492,3933.789474,3933.789474,1966.894737,1,13,nan,nan,...,33.97,48.79,57.68,45.30,28.87,16.22,10.19,8.39,6.78,230
10,gabriel peri,3790573.0,0.045648,9837.842105,9837.842105,4918.921053,1,13,nan,nan,...,31.95,34.54,38.88,38.49,29.48,18.40,11.34,7.23,5.40,64




Tableau: ligne numéro[14]


,Station,Trafic,Part du trafic de la station par rapport à toute la ligne,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nb_correspondance,Correspondance_1,Correspondance_2,Correspondance_3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,gare de lyon,9546825.0,0.172042,66037.947368,22012.649123,11006.324561,3,1,14,,...,34.76,45.22,57.99,60.21,38.73,27.60,17.40,10.24,4.80,3
2,bercy,1942106.0,0.034998,9023.947368,4511.973684,2255.986842,2,6,14,nan,...,2.37,109.81,104.32,67.23,40.18,13.57,0.11,14.23,0.09,61
4,madeleine,1776976.0,0.032023,12904.421053,4301.473684,2150.736842,3,8,12,14,...,43.85,53.91,65.73,71.49,60.81,23.76,11.64,11.65,10.10,27
5,porte de clichy,2639248.5,0.047561,12615.105263,6307.552632,3153.776316,2,13,14,nan,...,67.67,75.58,92.43,88.85,71.86,46.94,32.62,24.38,18.66,29
6,cour saint-emilion,2985122.0,0.053794,3173.444444,3173.444444,1586.722222,1,14,nan,nan,...,36.59,49.66,61.54,60.91,45.62,29.16,25.69,15.63,12.76,110
7,bibliotheque,11104474.0,0.200112,6088.000000,6088.000000,3044.000000,1,14,nan,nan,...,35.33,43.36,54.02,56.83,44.83,26.99,18.84,11.58,8.76,6
8,chatelet,1670158.8,0.030098,87779.526316,17555.905263,8777.952632,5,1,4,7,...,72.79,92.19,114.31,122.78,105.60,70.85,51.84,47.85,44.60,10
9,olympiades,5214595.0,0.093971,12997.142857,12997.142857,6498.571429,1,14,nan,nan,...,38.44,42.90,47.26,41.98,31.78,20.50,13.19,6.90,4.47,31
10,saint-lazare,6625676.8,0.119400,127163.368421,25432.673684,12716.336842,5,3,9,12,...,29.05,39.08,57.36,63.81,51.34,34.47,21.65,16.09,13.86,2
12,mairie de saint-ouen,2415405.0,0.043528,12017.736842,6008.868421,3004.434211,2,13,14,nan,...,67.36,77.62,93.80,89.63,63.45,40.80,24.30,16.35,13.66,39


In [208]:
# Définir le nouvel ordre des colonnes
nouvel_ordre_colonnes = [
    'Station',
    'Trafic', 
    'Part du trafic de la station par rapport à toute la ligne',
    'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station',
    'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne',
    'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction', 'nb_correspondance',
    'Correspondance_1',
    'Correspondance_2',
    'Correspondance_3',
    'Correspondance_4',
    'Correspondance_5',
    'nombre_validations_totales_pour_la_ligne','fréquentation_totale_ligne',
    '00:00-01:00',
    '01:00-02:00',
    '02:00-03:00',
    '03:00-04:00',
    '04:00-05:00',
    '05:00-06:00',
    '06:00-07:00',
    '07:00-08:00',
    '08:00-09:00',
    '09:00-10:00',
    '10:00-11:00',
    '11:00-12:00',
    '12:00-13:00',
    '13:00-14:00',
    '14:00-15:00',
    '15:00-16:00',
    '16:00-17:00',
    '17:00-18:00',
    '18:00-19:00',
    '19:00-20:00',
    '20:00-21:00',
    '21:00-22:00',
    '22:00-23:00',
    '23:00-00:00',
    'Rang'
]

# Appliquer le nouvel ordre des colonnes à chaque tableau
for tableau_nom, tableau in tableaux.items():
    tableaux[tableau_nom] = tableau[nouvel_ordre_colonnes]

# Vérifier que rien n'a été oublié
print("Colonnes disponibles :")


Colonnes disponibles :


In [209]:
# Nouveaux noms des colonnes
nouveaux_noms_colonnes = {
    'Station': 'Station',
    'Trafic': 'Trafic annuel dans la station',
    'Part du trafic de la station par rapport à toute la ligne': 'Part de la station dans le trafic annuel de la ligne',
    'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station': 'Moyenne de la fréquentation hebdomaire de la station en 2022, sur la période des JO (24-07-2022 au 11-08-2022)',
    'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne': 'Moyenne de la fréquentation hebdomadaire de chaque correspondance de la station en 2022, sur la période des JO',
    'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction': 'Moyenne de la fréquentation hebdomadaire de chaque correspondance, pour chaque direction en 2022, sur la période des JO',
    'Correspondance_1': 'Correspondance 1',
    'Correspondance_2': 'Correspondance 2',
    'Correspondance_3': 'Correspondance 3',
    'Correspondance_4': 'Correspondance 4',
    'Correspondance_5': 'Correspondance 5',
    'nb_correspondance': 'Nombre de correspondances de la station',
    'nombre_validations_totales_pour_la_ligne': 'Nombre de validations annuelles sur la ligne',
    'fréquentation_totale_ligne': 'Fréquentation totale de la ligne',
    '00:00-01:00': '00:00-01:00',
    '01:00-02:00': '01:00-02:00',
    '02:00-03:00': '02:00-03:00',
    '03:00-04:00': '03:00-04:00',
    '04:00-05:00': '04:00-05:00',
    '05:00-06:00': '05:00-06:00',
    '06:00-07:00': '06:00-07:00',
    '07:00-08:00': '07:00-08:00',
    '08:00-09:00': '08:00-09:00',
    '09:00-10:00': '09:00-10:00',
    '10:00-11:00': '10:00-11:00',
    '11:00-12:00': '11:00-12:00',
    '12:00-13:00': '12:00-13:00',
    '13:00-14:00': '13:00-14:00',
    '14:00-15:00': '14:00-15:00',
    '15:00-16:00': '15:00-16:00',
    '16:00-17:00': '16:00-17:00',
    '17:00-18:00': '17:00-18:00',
    '18:00-19:00': '18:00-19:00',
    '19:00-20:00': '19:00-20:00',
    '20:00-21:00': '20:00-21:00',
    '21:00-22:00': '21:00-22:00',
    '22:00-23:00': '22:00-23:00',
    '23:00-00:00': '23:00-00:00',
    'Rang': 'Rang'
}

# Afficher chaque tableau final avec le nouvel ordre des colonnes et les nouveaux noms
for tableau_nom, tableau in tableaux.items():
    print(f"Tableau: {tableau_nom}")
    
    # Réorganiser les colonnes selon le nouvel ordre
    tableau = tableau[nouvel_ordre_colonnes]
    
    # Renommer les colonnes
    tableau = tableau.rename(columns=nouveaux_noms_colonnes)
    
    display(tableau)
    print("\n" + "=" * 50 + "\n")


Tableau: ligne numéro[1]


,Station,Trafic annuel dans la station,Part de la station dans le trafic annuel de la ligne,"Moyenne de la fréquentation hebdomaire de la station en 2022, sur la période des JO (24-07-2022 au 11-08-2022)","Moyenne de la fréquentation hebdomadaire de chaque correspondance de la station en 2022, sur la période des JO","Moyenne de la fréquentation hebdomadaire de chaque correspondance, pour chaque direction en 2022, sur la période des JO",Nombre de correspondances de la station,Correspondance 1,Correspondance 2,Correspondance 3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,gare de lyon,9546825.0,0.031973,66037.947368,22012.649123,11006.324561,3,1,14,,...,34.76,45.22,57.99,60.21,38.73,27.60,17.40,10.24,4.80,3
2,belleville,3657219.0,0.012248,16611.631579,8305.815789,4152.907895,2,2,11,nan,...,38.84,39.88,41.62,37.49,32.01,21.30,14.72,13.35,11.97,12
3,saint-mande-tourelle,3944640.0,0.013211,8727.684211,8727.684211,4363.842105,1,1,nan,nan,...,35.69,37.54,44.98,47.84,34.26,19.04,10.38,8.05,5.99,58
4,bercy,1942106.0,0.006504,9023.947368,4511.973684,2255.986842,2,6,14,nan,...,2.37,109.81,104.32,67.23,40.18,13.57,0.11,14.23,0.09,61
6,saint-denis-universite,3569990.0,0.011956,8049.368421,8049.368421,4024.684211,1,13,nan,nan,...,30.33,28.90,31.51,27.77,18.64,13.60,8.89,6.17,4.36,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,michel-ange-auteuil,756025.0,0.002532,2899.631579,1449.815789,724.907895,2,9,10,nan,...,37.63,40.86,43.35,40.46,35.29,21.50,11.78,11.57,9.71,234
124,notre-dame-des-champs,1487256.0,0.004981,1661.578947,1661.578947,830.789474,1,12,nan,nan,...,36.70,46.97,51.64,51.20,38.02,25.48,14.76,14.98,12.61,238
125,malakoff-rue etienne dolet,1450451.0,0.004858,3242.105263,3242.105263,1621.052632,1,13,nan,nan,...,33.51,34.59,43.10,41.24,28.00,15.65,8.94,6.14,4.88,239
126,michel-ange-molitor,710276.0,0.002379,2581.315789,1290.657895,645.328947,2,9,10,nan,...,37.02,39.89,42.74,38.67,35.84,19.83,11.89,10.72,13.92,243




Tableau: ligne numéro[2]


,Station,Trafic annuel dans la station,Part de la station dans le trafic annuel de la ligne,"Moyenne de la fréquentation hebdomaire de la station en 2022, sur la période des JO (24-07-2022 au 11-08-2022)","Moyenne de la fréquentation hebdomadaire de chaque correspondance de la station en 2022, sur la période des JO","Moyenne de la fréquentation hebdomadaire de chaque correspondance, pour chaque direction en 2022, sur la période des JO",Nombre de correspondances de la station,Correspondance 1,Correspondance 2,Correspondance 3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,belleville,3.657219e+06,0.031606,16611.631579,8305.815789,4152.907895,2,2,11,nan,...,38.84,39.88,41.62,37.49,32.01,21.30,14.72,13.35,11.97,12
1,colonel fabien,3.043606e+06,0.026303,6090.315789,6090.315789,3045.157895,1,2,nan,nan,...,35.77,39.02,45.10,41.33,34.81,22.89,15.25,12.20,10.86,104
2,menilmontant,2.847264e+06,0.024606,5824.315789,5824.315789,2912.157895,1,2,nan,nan,...,32.69,34.55,37.85,37.05,33.65,24.10,18.15,17.53,17.79,118
3,blanche,2.167570e+06,0.018732,4236.421053,4236.421053,2118.210526,1,2,nan,nan,...,34.19,37.46,41.29,41.39,36.88,27.88,21.15,27.09,24.68,163
4,rue du bac,1.423364e+06,0.012301,2642.842105,2642.842105,1321.421053,1,12,nan,nan,...,40.07,50.13,57.77,57.82,48.87,23.13,15.46,12.09,11.93,242
5,philippe auguste,1.262653e+06,0.010912,2473.842105,2473.842105,1236.921053,1,2,nan,nan,...,36.68,41.25,49.12,42.83,32.67,21.55,13.32,12.08,9.82,256
6,monceau,1.148677e+06,0.009927,2529.789474,2529.789474,1264.894737,1,2,nan,nan,...,39.02,48.87,62.67,65.78,47.89,22.54,11.59,7.52,5.00,268
7,rennes,8.613340e+05,0.007444,1546.052632,1546.052632,773.026316,1,12,nan,nan,...,44.08,52.01,58.20,56.48,44.05,23.63,10.44,10.24,8.72,288
8,montparnasse-bienvenue,5.101806e+06,0.044090,67112.684211,16778.171053,8389.085526,4,4,6,12,...,26.18,36.84,55.50,62.72,49.65,31.05,21.54,16.83,13.49,4
10,madeleine,1.776976e+06,0.015357,12904.421053,4301.473684,2150.736842,3,8,12,14,...,43.85,53.91,65.73,71.49,60.81,23.76,11.64,11.65,10.10,27




Tableau: ligne numéro[3]


...

,Station,Trafic annuel dans la station,Part de la station dans le trafic annuel de la ligne,"Moyenne de la fréquentation hebdomaire de la station en 2022, sur la période des JO (24-07-2022 au 11-08-2022)","Moyenne de la fréquentation hebdomadaire de chaque correspondance de la station en 2022, sur la période des JO","Moyenne de la fréquentation hebdomadaire de chaque correspondance, pour chaque direction en 2022, sur la période des JO",Nombre de correspondances de la station,Correspondance 1,Correspondance 2,Correspondance 3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,rue du bac,1.423364e+06,0.023459,2642.842105,2642.842105,1321.421053,1,12,nan,nan,...,40.07,50.13,57.77,57.82,48.87,23.13,15.46,12.09,11.93,242
1,rennes,8.613340e+05,0.014196,1546.052632,1546.052632,773.026316,1,12,nan,nan,...,44.08,52.01,58.20,56.48,44.05,23.63,10.44,10.24,8.72,288
2,montparnasse-bienvenue,5.101806e+06,0.084085,67112.684211,16778.171053,8389.085526,4,4,6,12,...,26.18,36.84,55.50,62.72,49.65,31.05,21.54,16.83,13.49,4
4,madeleine,1.776976e+06,0.029287,12904.421053,4301.473684,2150.736842,3,8,12,14,...,43.85,53.91,65.73,71.49,60.81,23.76,11.64,11.65,10.10,27
5,sevres-babylone,1.696252e+06,0.027957,6540.105263,3270.052632,1635.026316,2,10,12,nan,...,39.67,51.13,59.22,66.54,57.85,35.27,12.25,9.71,8.96,88
6,mairie d'issy,2.874138e+06,0.047370,6616.526316,6616.526316,3308.263158,1,12,nan,nan,...,34.97,36.23,38.97,38.91,30.74,16.37,8.48,6.66,5.20,115
7,notre-dame-de-lorette,1.893828e+06,0.031213,3949.578947,3949.578947,1974.789474,1,12,nan,nan,...,35.01,35.65,44.18,49.69,39.57,25.65,17.51,18.02,18.33,188
8,porte de versailles,3.268157e+06,0.053864,15023.947368,15023.947368,7511.973684,1,12,nan,nan,...,39.11,50.70,60.08,62.00,53.19,32.02,19.18,19.01,18.49,93
9,vaugirard,2.482886e+06,0.040921,4768.736842,4768.736842,2384.368421,1,12,nan,nan,...,37.27,41.28,44.22,41.04,34.89,19.16,10.52,9.18,7.66,136
10,marx dormoy,2.425928e+06,0.039983,5344.315789,5344.315789,2672.157895,1,12,nan,nan,...,36.82,37.68,43.01,38.88,31.06,21.78,14.49,10.47,8.05,139




Tableau: ligne numéro[13]


,Station,Trafic annuel dans la station,Part de la station dans le trafic annuel de la ligne,"Moyenne de la fréquentation hebdomaire de la station en 2022, sur la période des JO (24-07-2022 au 11-08-2022)","Moyenne de la fréquentation hebdomadaire de chaque correspondance de la station en 2022, sur la période des JO","Moyenne de la fréquentation hebdomadaire de chaque correspondance, pour chaque direction en 2022, sur la période des JO",Nombre de correspondances de la station,Correspondance 1,Correspondance 2,Correspondance 3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,saint-denis-universite,3569990.0,0.042992,8049.368421,8049.368421,4024.684211,1,13,nan,nan,...,30.33,28.90,31.51,27.77,18.64,13.60,8.89,6.17,4.36,74
1,les courtilles,2802689.0,0.033751,8152.789474,8152.789474,4076.394737,1,13,nan,nan,...,30.03,31.76,35.24,31.76,23.70,16.18,10.77,7.34,4.88,122
2,porte de saint-ouen,2710638.0,0.032643,6262.368421,6262.368421,3131.184211,1,13,nan,nan,...,34.06,38.01,42.89,40.38,34.53,23.47,16.62,12.24,9.27,125
3,montparnasse-bienvenue,5101806.0,0.061438,67112.684211,16778.171053,8389.085526,4,4,6,12,...,26.18,36.84,55.50,62.72,49.65,31.05,21.54,16.83,13.49,4
5,porte de clichy,2639248.5,0.031783,12615.105263,6307.552632,3153.776316,2,13,14,nan,...,67.67,75.58,92.43,88.85,71.86,46.94,32.62,24.38,18.66,29
6,place clichy,2580966.0,0.031081,10922.736842,5461.368421,2730.684211,2,2,13,nan,...,34.37,37.23,41.68,40.48,38.04,26.06,20.67,21.62,21.26,33
7,porte de vanves,3395358.0,0.040889,7711.315789,7711.315789,3855.657895,1,13,nan,nan,...,32.51,35.64,41.18,39.19,30.95,21.34,13.95,10.18,8.11,87
8,pernety,2173567.0,0.026175,4404.263158,4404.263158,2202.131579,1,13,nan,nan,...,33.91,33.31,34.66,34.29,31.53,21.05,13.67,12.71,10.96,162
9,carrefour pleyel,1535528.0,0.018492,3933.789474,3933.789474,1966.894737,1,13,nan,nan,...,33.97,48.79,57.68,45.30,28.87,16.22,10.19,8.39,6.78,230
10,gabriel peri,3790573.0,0.045648,9837.842105,9837.842105,4918.921053,1,13,nan,nan,...,31.95,34.54,38.88,38.49,29.48,18.40,11.34,7.23,5.40,64




Tableau: ligne numéro[14]


,Station,Trafic annuel dans la station,Part de la station dans le trafic annuel de la ligne,"Moyenne de la fréquentation hebdomaire de la station en 2022, sur la période des JO (24-07-2022 au 11-08-2022)","Moyenne de la fréquentation hebdomadaire de chaque correspondance de la station en 2022, sur la période des JO","Moyenne de la fréquentation hebdomadaire de chaque correspondance, pour chaque direction en 2022, sur la période des JO",Nombre de correspondances de la station,Correspondance 1,Correspondance 2,Correspondance 3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,gare de lyon,9546825.0,0.172042,66037.947368,22012.649123,11006.324561,3,1,14,,...,34.76,45.22,57.99,60.21,38.73,27.60,17.40,10.24,4.80,3
2,bercy,1942106.0,0.034998,9023.947368,4511.973684,2255.986842,2,6,14,nan,...,2.37,109.81,104.32,67.23,40.18,13.57,0.11,14.23,0.09,61
4,madeleine,1776976.0,0.032023,12904.421053,4301.473684,2150.736842,3,8,12,14,...,43.85,53.91,65.73,71.49,60.81,23.76,11.64,11.65,10.10,27
5,porte de clichy,2639248.5,0.047561,12615.105263,6307.552632,3153.776316,2,13,14,nan,...,67.67,75.58,92.43,88.85,71.86,46.94,32.62,24.38,18.66,29
6,cour saint-emilion,2985122.0,0.053794,3173.444444,3173.444444,1586.722222,1,14,nan,nan,...,36.59,49.66,61.54,60.91,45.62,29.16,25.69,15.63,12.76,110
7,bibliotheque,11104474.0,0.200112,6088.000000,6088.000000,3044.000000,1,14,nan,nan,...,35.33,43.36,54.02,56.83,44.83,26.99,18.84,11.58,8.76,6
8,chatelet,1670158.8,0.030098,87779.526316,17555.905263,8777.952632,5,1,4,7,...,72.79,92.19,114.31,122.78,105.60,70.85,51.84,47.85,44.60,10
9,olympiades,5214595.0,0.093971,12997.142857,12997.142857,6498.571429,1,14,nan,nan,...,38.44,42.90,47.26,41.98,31.78,20.50,13.19,6.90,4.47,31
10,saint-lazare,6625676.8,0.119400,127163.368421,25432.673684,12716.336842,5,3,9,12,...,29.05,39.08,57.36,63.81,51.34,34.47,21.65,16.09,13.86,2
12,mairie de saint-ouen,2415405.0,0.043528,12017.736842,6008.868421,3004.434211,2,13,14,nan,...,67.36,77.62,93.80,89.63,63.45,40.80,24.30,16.35,13.66,39


### PROBLÈMES À RÉGLER : METTRE LES COLONNES DE CHAQUE TABLEAU DANS UN MEILLEUR ORDRE ET RENOMMER LES COLONNES POUR QUE CE SOIT + JOLI ### 

Trafic = le trafic annuel dans la station ; nb_correspondances = le nb de lignes dans la station ; fréquentation totale ligne = trafic annuel / nb_correspondances ; part du trafic de la station par rapport à toute la ligne = trafic / fréquentation_totale_ligne ; les titres de colonnes avec période_jo_2022 : moyenne hebdomadaire sur la période du 24-07-2022 au 11-08-2022

### TABLEAU A EXPLOITER CI DESSUS ###

In [46]:
# Créer le dictionnaire pour les nouvelles variables
tableaux_pour_variables = {}

for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Supprimer les colonnes spécifiées
    columns_to_drop = ['Rang', 'Trafic', 'nb_correspondance', 'fréquentation_totale_ligne', 
                       'Part du trafic de la station par rapport à toute la ligne',
                       'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station']

    # Créer un nouveau tableau sans les colonnes spécifiées
    new_table = current_table.drop(columns=columns_to_drop)

    # Ajouter le nouveau tableau dans le dictionnaire
    tableaux[f'ligne numéro[{k_str}]'] = new_table

    # Afficher le tableau mis à jour
    print(f"Tableau 'ligne numéro[{k_str}]' sans les colonnes spécifiées :")
    display(tableaux[f'ligne numéro[{k_str}]'])


Tableau 'ligne numéro[1]' sans les colonnes spécifiées :


,Station,00:00-01:00,01:00-02:00,02:00-03:00,03:00-04:00,04:00-05:00,05:00-06:00,06:00-07:00,07:00-08:00,08:00-09:00,...,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nombre_validations_totales_pour_la_ligne
0,la defense,7.07,1.51,0.01,0.00,0.92,7.37,15.51,32.98,57.04,...,103.14,108.83,84.18,53.23,26.30,14.93,14.40,15598.768421,7799.384211,138177.887449
1,hotel de ville,12.16,4.87,0.09,0.01,0.00,1.58,2.33,4.42,9.79,...,60.08,62.00,53.19,32.02,19.18,19.01,18.49,3004.789474,1502.394737,138177.887449
2,palais-royal,9.30,2.52,0.09,0.00,0.00,1.09,1.31,2.67,5.65,...,70.75,67.08,42.63,27.29,21.24,20.44,20.58,2436.673684,1218.336842,138177.887449
3,concorde,10.30,2.87,0.08,0.04,0.02,2.15,1.53,3.05,5.02,...,66.58,69.70,54.94,27.15,19.30,18.36,17.88,1927.305263,963.652632,138177.887449
4,argentine,6.64,2.50,0.16,0.00,0.00,2.29,5.23,11.44,28.23,...,41.96,46.19,39.54,23.87,13.05,10.02,9.98,879.326316,439.663158,138177.887449
5,champs-elysees-clemenceau,15.47,5.89,0.09,0.00,0.02,3.31,4.93,4.65,5.40,...,62.33,67.18,52.39,31.77,25.55,24.32,23.10,860.263158,430.131579,138177.887449
6,tuileries,10.76,2.14,0.03,0.00,0.01,0.88,1.26,3.22,7.74,...,59.73,61.08,51.92,32.26,25.46,23.30,22.92,938.063158,469.031579,138177.887449
7,bastille,18.39,8.33,0.40,0.06,0.03,4.99,4.52,6.83,15.09,...,49.12,48.68,39.54,24.82,22.50,28.78,25.92,3159.147368,1579.573684,138177.887449
8,franklin d. roosevelt,12.37,3.18,0.04,0.00,0.00,2.07,2.67,3.24,6.12,...,57.97,65.83,58.93,37.06,25.12,20.40,17.47,2646.378947,1323.189474,138177.887449
9,nation,15.32,3.51,0.30,0.04,0.35,9.99,12.07,27.31,55.11,...,90.17,88.67,79.27,49.96,33.56,27.09,22.85,3893.031579,1946.515789,138177.887449


Tableau 'ligne numéro[2]' sans les colonnes spécifiées :


,Station,00:00-01:00,01:00-02:00,02:00-03:00,03:00-04:00,04:00-05:00,05:00-06:00,06:00-07:00,07:00-08:00,08:00-09:00,...,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nombre_validations_totales_pour_la_ligne
0,victor hugo,7.13,1.43,0.03,0.00,0.00,2.01,3.47,8.66,21.72,...,48.86,48.08,43.57,22.99,12.69,12.26,12.33,1056.842105,528.421053,70635.936842
1,alexandre dumas,6.21,2.27,0.09,0.00,0.00,3.21,7.42,17.03,32.36,...,43.01,39.28,33.30,22.87,14.42,12.51,10.12,929.863158,464.931579,70635.936842
2,nation,15.32,3.51,0.30,0.04,0.35,9.99,12.07,27.31,55.11,...,90.17,88.67,79.27,49.96,33.56,27.09,22.85,3893.031579,1946.515789,70635.936842
3,barbes-rochechouart,5.36,2.10,0.07,0.00,0.00,2.29,4.46,10.37,19.10,...,45.75,44.03,37.70,22.18,14.09,11.06,8.63,2387.810526,1193.905263,70635.936842
4,villiers,6.53,1.88,0.10,0.01,0.01,1.46,3.40,9.91,25.03,...,43.72,45.97,42.67,26.71,14.65,12.93,11.77,1310.957895,655.478947,70635.936842
5,pigalle,19.27,8.09,0.09,0.00,0.00,4.63,6.68,10.88,21.03,...,39.98,40.12,36.95,26.15,21.85,28.69,26.75,1318.252632,659.126316,70635.936842
6,anvers,10.19,3.06,0.02,0.00,0.02,1.60,3.44,9.07,20.61,...,48.60,42.74,37.55,24.20,19.35,25.98,19.82,1303.031579,651.515789,70635.936842
7,ternes,6.72,1.92,0.08,0.01,0.01,1.36,3.23,8.58,18.47,...,46.53,48.19,45.57,24.19,13.65,14.27,14.99,966.526316,483.263158,70635.936842
8,courcelles,4.81,1.88,0.08,0.00,0.00,1.87,4.29,10.48,22.27,...,53.72,52.95,45.88,25.35,11.32,9.70,8.31,632.736842,316.368421,70635.936842
9,funiculaire,5.63,1.22,NaN,0.00,NaN,0.35,3.73,6.53,8.72,...,47.08,42.86,37.92,31.95,27.11,20.99,13.78,894.263158,447.131579,70635.936842


Tableau 'ligne numéro[3]' sans les colonnes spécifiées :


...

,Station,00:00-01:00,01:00-02:00,02:00-03:00,03:00-04:00,04:00-05:00,05:00-06:00,06:00-07:00,07:00-08:00,08:00-09:00,...,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nombre_validations_totales_pour_la_ligne
0,concorde,10.30,2.87,0.08,0.04,0.02,2.15,1.53,3.05,5.02,...,66.58,69.70,54.94,27.15,19.30,18.36,17.88,1927.305263,963.652632,105667.292763
1,pasteur,7.42,2.12,0.13,0.00,0.00,2.47,6.30,15.74,32.86,...,43.50,44.12,35.49,20.42,12.83,11.10,10.56,1261.547368,630.773684,105667.292763
2,abbesses,10.18,3.20,0.02,0.00,NaN,1.31,4.34,10.27,24.71,...,45.59,43.24,37.00,23.36,19.02,19.36,18.35,455.463158,227.731579,105667.292763
3,trinite-d'estienne d'orves,12.18,6.56,0.22,0.00,0.00,1.10,2.77,6.27,13.19,...,51.19,54.76,49.49,33.64,26.85,21.44,16.85,690.957895,345.478947,105667.292763
4,saint-georges,9.83,3.71,0.10,0.00,0.01,2.00,5.94,11.59,24.44,...,47.79,42.88,34.41,21.11,16.17,26.32,17.00,289.821053,144.910526,105667.292763
5,assemblee nationale,14.18,8.04,0.10,0.01,0.00,3.03,3.32,6.54,13.03,...,65.30,64.65,41.95,25.79,16.80,15.93,18.23,320.452632,160.226316,105667.292763
6,saint-lazare,8.62,0.10,0.01,0.01,0.12,3.15,9.60,20.07,20.19,...,57.36,63.81,51.34,34.47,21.65,16.09,13.86,25432.673684,12716.336842,105667.292763
7,saint-lazare,8.06,1.80,0.14,0.04,0.08,8.17,21.66,48.06,74.66,...,85.45,92.97,77.22,51.26,27.60,21.59,14.81,25432.673684,12716.336842,105667.292763
8,marcadet-poissonniers,4.49,1.81,0.10,0.00,0.00,3.48,8.00,16.62,27.65,...,43.07,42.94,37.58,27.00,16.68,10.82,7.56,1883.484211,941.742105,105667.292763
9,convention,4.27,1.32,0.07,0.00,0.00,2.02,6.01,17.82,37.66,...,37.75,38.46,33.51,20.97,13.06,9.80,7.74,1431.431579,715.715789,105667.292763


Tableau 'ligne numéro[13]' sans les colonnes spécifiées :


,Station,00:00-01:00,01:00-02:00,02:00-03:00,03:00-04:00,04:00-05:00,05:00-06:00,06:00-07:00,07:00-08:00,08:00-09:00,...,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nombre_validations_totales_pour_la_ligne
0,basilique de saint-denis,1.98,0.30,0.05,0.04,0.73,5.35,9.40,31.38,58.45,...,39.92,33.92,25.82,14.12,8.43,6.61,4.61,377.852632,188.926316,37710.673684
1,plaisance,4.13,1.31,0.03,0.00,0.00,5.80,10.04,22.39,38.50,...,39.72,38.27,32.66,22.15,12.53,8.94,7.01,1475.747368,737.873684,37710.673684
2,malakoff-plateau de vanves,2.93,1.02,0.09,0.00,0.00,3.69,9.44,21.46,41.12,...,39.62,41.65,31.21,18.21,9.74,7.87,5.92,929.536842,464.768421,37710.673684
3,guy moquet,5.08,2.09,0.06,0.00,0.00,2.90,8.39,19.10,37.29,...,37.81,36.78,33.77,20.83,12.24,9.86,8.41,1075.715789,537.857895,37710.673684
4,champs-elysees-clemenceau,15.47,5.89,0.09,0.00,0.02,3.31,4.93,4.65,5.40,...,62.33,67.18,52.39,31.77,25.55,24.32,23.10,860.263158,430.131579,37710.673684
5,les agnettes,2.65,0.77,0.04,0.00,0.00,9.27,18.41,34.29,49.12,...,34.57,31.37,25.58,17.10,10.05,7.16,5.00,824.663158,412.331579,37710.673684
6,la fourche,7.98,3.66,0.13,0.00,0.00,3.29,8.11,16.52,31.78,...,40.81,40.02,37.61,24.24,15.98,12.67,11.38,693.610526,346.805263,37710.673684
7,varenne,6.08,2.24,0.09,0.00,0.00,1.05,3.29,8.41,12.98,...,69.74,68.24,34.46,17.61,11.87,10.16,16.57,369.021053,184.510526,37710.673684
8,chatillon-montrouge,2.28,0.61,0.02,0.00,0.00,4.92,12.21,27.89,48.67,...,41.77,41.56,29.17,16.87,9.27,6.52,5.61,2334.905263,1167.452632,37710.673684
9,mairie de saint-ouen,7.75,2.71,0.19,0.01,0.01,11.46,26.18,51.14,66.81,...,93.80,89.63,63.45,40.80,24.30,16.35,13.66,2403.547368,1201.773684,37710.673684


Tableau 'ligne numéro[14]' sans les colonnes spécifiées :


,Station,00:00-01:00,01:00-02:00,02:00-03:00,03:00-04:00,04:00-05:00,05:00-06:00,06:00-07:00,07:00-08:00,08:00-09:00,...,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nombre_validations_totales_pour_la_ligne
0,mairie de saint-ouen,7.75,2.71,0.19,0.01,0.01,11.46,26.18,51.14,66.81,...,93.80,89.63,63.45,40.80,24.30,16.35,13.66,2403.547368,1201.773684,47415.801671
1,pont cardinet,3.67,1.64,0.11,0.00,0.00,2.92,7.24,16.72,38.78,...,43.22,45.32,37.99,23.12,14.08,7.68,6.32,1896.652632,948.326316,47415.801671
2,pyramides,6.84,2.03,0.13,0.01,0.01,0.95,1.94,3.70,7.64,...,63.20,67.67,51.92,33.49,24.30,22.48,18.01,1711.652632,855.826316,47415.801671
3,clichy saint-ouen,2.59,1.24,0.18,0.01,0.00,4.59,10.40,19.34,30.42,...,54.71,54.96,36.11,20.36,12.13,6.13,4.06,1823.568421,911.784211,47415.801671
4,bibliotheque,6.46,1.35,0.06,0.00,0.10,5.02,9.13,13.29,19.76,...,54.02,56.83,44.83,26.99,18.84,11.58,8.76,1217.600000,608.800000,47415.801671
5,olympiades,2.74,1.23,0.06,0.00,0.01,6.71,10.26,17.73,32.47,...,47.26,41.98,31.78,20.50,13.19,6.90,4.47,2599.428571,1299.714286,47415.801671
6,gare de lyon,2.16,0.01,0.00,NaN,0.34,1.20,10.05,22.63,19.75,...,57.99,60.21,38.73,27.60,17.40,10.24,4.80,13207.589474,6603.794737,47415.801671
7,gare de lyon,18.38,4.31,0.30,0.10,0.62,8.89,19.53,38.87,63.64,...,84.89,89.26,74.77,56.04,41.78,38.76,34.89,13207.589474,6603.794737,47415.801671
8,bercy,NaN,NaN,NaN,NaN,NaN,6.98,10.94,2.74,30.49,...,104.32,67.23,40.18,13.57,0.11,14.23,0.09,1804.789474,902.394737,47415.801671
9,bercy,10.59,2.39,0.22,0.02,0.30,3.90,8.60,17.20,24.96,...,43.36,44.68,36.17,35.22,24.11,25.58,36.50,1804.789474,902.394737,47415.801671


In [47]:
for k in numéro:
    k_str = str(k)
    tableaux[f'ligne numéro[{k_str}]'].to_csv(f'ligne_numero_{k_str}.csv', index=False) 

In [49]:
def affluence_par_heure(horaire, station, line):
    # Obtenir le tableau correspondant à la ligne
    line_table = tableaux[f'ligne numéro[{line}]']

    # Extraire l'heure de l'horaire entré par l'utilisateur
    hour, minute = map(int, horaire.split(':'))

    # Calculer l'indice de la colonne correspondant à la plage horaire donnée
    time_range_index = (hour * 60 + minute) // 60

    # Vérifier si l'indice est dans les limites du tableau
    if time_range_index < 0 or time_range_index >= len(line_table.columns):
        raise ValueError(f"Invalid time: {horaire}")

    # Filtrer le tableau pour la station donnée
    station_row = line_table[line_table['Station'] == station]

    if station_row.empty:
        raise ValueError(f"La station {station} n'est pas sur la ligne numéro {line}")

    # Récupérer la valeur dans la colonne de la plage horaire et arrondir à l'entier le plus proche
    affluence_value = round(station_row.iloc[0, time_range_index])

    return affluence_value

# Exemple d'utilisation
line_number = "1"
station_name = "bastille"
time = "9:00"

try:
    affluence_value = affluence_par_heure(time, station_name, line_number)
    print(f"L'affluence à la station {station_name}, sur la ligne numéro {line_number}, à {time} est de : {affluence_value} personnes sur une heure")
except ValueError as e:
    print(e)


L'affluence à la station bastille, sur la ligne numéro 1, à 9:00 est de : 18 personnes sur une heure
